In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
import numpy as np

# Dataset Class
class EEGDataset(Dataset):
    def __init__(self, data_folder, outcome_file=None, target_length=600, use_labels=True):
        self.data_folder = data_folder
        self.target_length = target_length
        self.file_list = [f for f in os.listdir(data_folder) if f.endswith('.csv')]
        self.use_labels = use_labels
        self.outcome_dict = {}

        if use_labels and outcome_file is not None:
            self.outcome_data = pd.read_csv(outcome_file)
            self.outcome_dict = self.outcome_data.set_index('pat_ID')['outcome'].to_dict()
            self.file_list = [f for f in self.file_list if f.split('.')[0] in self.outcome_dict]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        filename = self.file_list[idx]
        patient_id = filename.split('.')[0]
        file_path = os.path.join(self.data_folder, filename)
        signal = self.load_data(file_path)
        processed_signal = self.padding(signal, self.target_length)

        label = -1
        if self.use_labels:
            label = 1 if self.outcome_dict.get(patient_id, 'Bad Outcome') == 'Good Outcome' else 0

        return torch.tensor(processed_signal, dtype=torch.float32).unsqueeze(0), label

    @staticmethod
    def load_data(file_path, column_name='BCI'):
        data = pd.read_csv(file_path)
        return data[column_name].values

    @staticmethod
    def padding(signal, target_length):
        if len(signal) < target_length:
            return np.pad(signal, (0, target_length - len(signal)), 'constant', constant_values=-1)
        return signal[:target_length]


# Siamese Network
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 32)

    def forward_one(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.mean(x, dim=-1)  # Global Average Pooling
        x = F.relu(self.fc1(x))
        return self.fc2(x)

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        return out1, out2


# Contrastive Loss
def contrastive_loss(out1, out2, label, margin=1.0):
    euclidean_distance = F.pairwise_distance(out1, out2)
    loss = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                      (label) * torch.pow(torch.clamp(margin - euclidean_distance, min=0.0), 2))
    return loss


# Training Loop
def train(model, dataloader, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for data in dataloader:
            signal1, label1 = data
            signal2, label2 = data  # Simple example; you can choose another sample for contrastive pairs

            # No .cuda() calls; everything stays on CPU
            label = torch.tensor([1 if label1 == label2 else 0], dtype=torch.float32)

            optimizer.zero_grad()
            out1, out2 = model(signal1, signal2)
            loss = contrastive_loss(out1, out2, label)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')


# Main
if __name__ == '__main__':

    data_folder = '5min_smoothed_data/'
    outcome_file = 'valid_patients_outcome.csv'
    dataset = EEGDataset(data_folder, outcome_file)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

    model = SiameseNetwork()  # No .cuda() here; runs on CPU
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train(model, dataloader, optimizer,epochs=100)

Epoch 1/100, Loss: 0.9999886751174927
Epoch 2/100, Loss: 0.9999886751174927
Epoch 3/100, Loss: 0.9999886751174927
Epoch 4/100, Loss: 0.9999886751174927
Epoch 5/100, Loss: 0.9999886751174927
Epoch 6/100, Loss: 0.9999886751174927
Epoch 7/100, Loss: 0.9999886751174927
Epoch 8/100, Loss: 0.9999886751174927
Epoch 9/100, Loss: 0.9999886751174927
Epoch 10/100, Loss: 0.9999886751174927
Epoch 11/100, Loss: 0.9999886751174927
Epoch 12/100, Loss: 0.9999886751174927
Epoch 13/100, Loss: 0.9999886751174927
Epoch 14/100, Loss: 0.9999886751174927
Epoch 15/100, Loss: 0.9999886751174927
Epoch 16/100, Loss: 0.9999886751174927
Epoch 17/100, Loss: 0.9999886751174927
Epoch 18/100, Loss: 0.9999886751174927
Epoch 19/100, Loss: 0.9999886751174927
Epoch 20/100, Loss: 0.9999886751174927
Epoch 21/100, Loss: 0.9999886751174927
Epoch 22/100, Loss: 0.9999886751174927
Epoch 23/100, Loss: 0.9999886751174927
Epoch 24/100, Loss: 0.9999886751174927
Epoch 25/100, Loss: 0.9999886751174927


KeyboardInterrupt: 

In [6]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.version.cuda)  # Shows the version of CUDA that PyTorch was built with (if available)

False
None
